In [1]:
import mysql.connector
import pandas as pd
from news_scraping import scraping
from nltk.corpus import stopwords
import gensim
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from fuzzywuzzy import process
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

C:\Users\Andre\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
C:\Users\Andre\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
scraping()

Iniciando scraping do site G1. Isto pode demorar um pouco...
Buscando notícias e salvando em lista...
Iniciando scraping do site Google News...
Buscando notícias e salvando em lista...
Iniciando scraping do site Estadão...
Buscando notícias e salvando em lista...
Importando listas para o banco de dados...
Pronto.
Processo finalizado!


In [3]:
db = mysql.connector.connect(
   host="database-1.cpqjjsrzpykc.us-east-2.rds.amazonaws.com",
   user="admin",
   passwd="Andre.2021",
   database="noticias"
)

print(db)

In [4]:
cursor = db.cursor()
query = ("SELECT * FROM noticias.news")
cursor.execute(query)
records = cursor.fetchall()
db.close()

In [31]:
records[0]

(1,
 'Governo usa foto de homem armado para celebrar o dia do agricultor',
 'Tuíte no perfil oficial da Secretaria de Comunicação recebeu críticas nas redes sociais.',
 'Há 3 horas',
 'https://g1.globo.com/economia/agronegocios/noticia/2021/07/28/governo-bolsonaro-publica-foto-de-homem-armado-para-parabenizar-o-dia-do-agricultor.ghtml',
 'https://s2.glbimg.com/3LCpgXP8fMzJRHUvuAzmhYzIZHc=/0x34:1025x611/540x304/smart/filters:max_age(3600)/https://i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2021/X/q/Dl2RiJSOm23AaDuVeaOg/secretaria-especial.jpg',
 'G1')

In [5]:
news_df = pd.DataFrame(records, columns=['id','titulo','subtitulo','hora','link','img', 'fonte'])
news_df = news_df.drop(columns=['id'])

In [6]:
news_df['titulo'].isnull().sum()

0

In [7]:
news_df.shape

(140, 6)

In [8]:
def process_texto(s):
    
    # seleciona apenas letras (lembrando que o texto está em português e as letras possuem acento)
    texto_limpo = gensim.utils.simple_preprocess(s)
    
    # remove stopwords
    sw = stopwords.words('portuguese')
    sem_stopwords = [word for word in texto_limpo if word not in sw]
    
    return sem_stopwords

In [9]:
news_df['titulo_sw'] = news_df['titulo'].apply(lambda s: process_texto(s)).apply(lambda x: ' '.join(x))

In [10]:
news_df.head(2)

,titulo,subtitulo,hora,link,img,fonte,titulo_sw
0,Governo usa foto de homem armado para celebrar...,Tuíte no perfil oficial da Secretaria de Comun...,Há 3 horas,https://g1.globo.com/economia/agronegocios/not...,https://s2.glbimg.com/3LCpgXP8fMzJRHUvuAzmhYzI...,G1,governo usa foto homem armado celebrar dia agr...
1,Concurso do Banco do Brasil prorroga prazo de ...,Processo seletivo oferece 4.480 vagas com salá...,Há 4 horas,https://g1.globo.com/economia/concursos-e-empr...,https://s2.glbimg.com/Ud-LAzmADnNTydrWj1K-Ah7A...,G1,concurso banco brasil prorroga prazo inscrição


In [11]:
# Bag-of-Words (bow) vectorises the data
# transforming tokens into useful features
     
bow_transformer = CountVectorizer().fit_transform(news_df['titulo_sw'])

news_tfidf = TfidfTransformer().fit_transform(bow_transformer)


In [12]:
nn = NearestNeighbors(metric='cosine', n_neighbors=6)

In [238]:
# Recomendador de notícias

lista = []
def recommender(noticia):
    lista.clear()
    nn.fit(news_tfidf)
    idx=process.extractOne(noticia, news_df['titulo'])[2]
    print('Notícias recomendadas: ')
    distances, indices=nn.kneighbors(news_tfidf[idx], n_neighbors=6)
    n=0
    for i in indices:
        lista.append(i)
        
    for n in lista[0][1:]:
        print(f'{news_df.iloc[n]} \n\n')
        

In [240]:
recommender(news_df['titulo'][43])

Notícias recomendadas: 
titulo       Estátua de Borba Gato em São Paulo causa polêm...
subtitulo                                                     
hora                                             6 horas atrás
link         news.google.com/articles/CBMie2h0dHBzOi8vd3d3M...
img          https://lh6.googleusercontent.com/proxy/WWih18...
fonte                                         Folha de S.Paulo
titulo_sw    estátua borba gato paulo causa polêmica desde ...
Name: 60, dtype: object 


titulo       VÍDEO: veja o golpe de Maria Portela que não f...
subtitulo                                                     
hora                                                Há 6 horas
link         https://g1.globo.com/rs/rio-grande-do-sul/vide...
img          https://s2.glbimg.com/LZTQhCZ9qWQAkJ-CDFLVUQ6c...
fonte                                                       G1
titulo_sw             vídeo veja golpe maria portela computado
Name: 8, dtype: object 


titulo       Veja dos gols da vitória do 